# Implementation Costs Analysis Example

This notebook demonstrates how to perform implementation costs analysis for health interventions.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define implementation cost analysis functions
def calculate_implementation_costs(strategy, patient_volume, time_horizon=5):
    """
    Calculate implementation costs for a specific strategy.
    
    Args:
        strategy: Treatment strategy name
        patient_volume: Projected annual patient volume
        time_horizon: Implementation time horizon in years
        
    Returns:
        Dictionary with implementation cost breakdown
    """
    # Define cost parameters by strategy
    strategy_params = {
        'ECT': {
            'startup_equipment': 100000,  # Cost of ECT machine and setup
            'training_per_provider': 5000,
            'providers_needed_per_100_patients': 2,
            'ongoing_equipment_cost_per_patient': 100,
            'additional_staff_cost_per_patient': 200
        },
        'IV-KA': {
            'startup_equipment': 50000,  # Cost of IV setup and monitoring equipment
            'training_per_provider': 7500,
            'providers_needed_per_100_patients': 3,  # More staff needed for IV procedures
            'ongoing_equipment_cost_per_patient': 150,
            'additional_staff_cost_per_patient': 250
        },
        'PO-KA': {
            'startup_equipment': 10000,  # Minimal equipment needed
            'training_per_provider': 3000,
            'providers_needed_per_100_patients': 1,  # Less staff needed for oral treatment
            'ongoing_equipment_cost_per_patient': 50,
            'additional_staff_cost_per_patient': 100
        }
    }
    
    if strategy not in strategy_params:
        raise ValueError(f"Unknown strategy: {strategy}")
    
    params = strategy_params[strategy]
    
    # Calculate providers needed
    providers_needed = np.ceil((patient_volume / 100) * params['providers_needed_per_100_patients'])
    
    # Calculate costs
    startup_equipment = params['startup_equipment']
    startup_training = providers_needed * params['training_per_provider']
    
    # Annual operational costs
    operational_equipment = patient_volume * params['ongoing_equipment_cost_per_patient']
    operational_staff = patient_volume * params['additional_staff_cost_per_patient']
    
    # Total costs
    total_startup = startup_equipment + startup_training
    total_5year_ops = (operational_equipment + operational_staff) * time_horizon
    total_cost = total_startup + total_5year_ops
    
    return {
        'strategy': strategy,
        'patient_volume': patient_volume,
        'providers_needed': providers_needed,
        'startup_equipment': startup_equipment,
        'startup_training': startup_training,
        'total_startup': total_startup,
        'annual_operational_equipment': operational_equipment,
        'annual_operational_staff': operational_staff,
        'annual_operational_total': operational_equipment + operational_staff,
        'total_5year_operational': (operational_equipment + operational_staff) * time_horizon,
        'total_implementation_cost': total_cost,
        'cost_per_patient_total': total_cost / patient_volume if patient_volume > 0 else 0,
        'cost_per_patient_annual': (operational_equipment + operational_staff) / patient_volume if patient_volume > 0 else 0
    }

In [ ]:
# Define strategies and volumes
strategies = ['ECT', 'IV-KA', 'PO-KA']
patient_volumes = [1200, 800, 1500]  # Annual patient volumes

# Calculate implementation costs for each strategy
impl_costs_data = []
for i, strategy in enumerate(strategies):
    costs = calculate_implementation_costs(strategy, patient_volumes[i])
    impl_costs_data.append(costs)

impl_costs_df = pd.DataFrame(impl_costs_data)
print(impl_costs_df)

In [ ]:
# Visualize implementation costs
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Total implementation cost by strategy
ax[0,0].bar(impl_costs_df['strategy'], impl_costs_df['total_implementation_cost'], 
            color=['blue', 'green', 'orange'], alpha=0.7)
ax[0,0].set_ylabel('Total Cost ($AUD)')
ax[0,0].set_title('Total Implementation Cost by Strategy')
ax[0,0].grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(impl_costs_df['total_implementation_cost']):
    ax[0,0].text(i, v + max(impl_costs_df['total_implementation_cost'])*0.01, 
                 f'${v:,.0f}', ha='center', va='bottom')

# 2. Cost breakdown for each strategy
bottom_start = np.zeros(len(impl_costs_df))
categories = ['Startup', 'Operational']
startups = impl_costs_df['total_startup'].values
operations = impl_costs_df['total_5year_operational'].values

ax[0,1].bar(impl_costs_df['strategy'], startups, label='Startup Costs', alpha=0.7, color='steelblue')
ax[0,1].bar(impl_costs_df['strategy'], operations, bottom=startups, label='5-Year Operational', 
            alpha=0.7, color='lightgreen')
ax[0,1].set_ylabel('Total Cost ($AUD)')
ax[0,1].set_title('Cost Breakdown: Startup vs Operational')
ax[0,1].legend()
ax[0,1].grid(True, alpha=0.3)

# 3. Cost per patient
ax[1,0].bar(impl_costs_df['strategy'], impl_costs_df['cost_per_patient_total'], 
            color=['blue', 'green', 'orange'], alpha=0.7)
ax[1,0].set_ylabel('Cost per Patient ($AUD)')
ax[1,0].set_title('Total Implementation Cost per Patient')
ax[1,0].grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(impl_costs_df['cost_per_patient_total']):
    ax[1,0].text(i, v + max(impl_costs_df['cost_per_patient_total'])*0.01, 
                 f'${v:,.0f}', ha='center', va='bottom')

# 4. Provider requirements
ax[1,1].bar(impl_costs_df['strategy'], impl_costs_df['providers_needed'], 
            color=['blue', 'green', 'orange'], alpha=0.7)
ax[1,1].set_ylabel('Number of Providers Needed')
ax[1,1].set_title('Staffing Requirements by Strategy')
ax[1,1].grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(impl_costs_df['providers_needed']):
    ax[1,1].text(i, v + max(impl_costs_df['providers_needed'])*0.01, 
                 f'{int(v)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Perform sensitivity analysis on patient volume
volume_range = np.arange(100, 2001, 100)  # From 100 to 2000 by 100
sensitivity_results = []

for vol in volume_range:
    for strategy in strategies:
        costs = calculate_implementation_costs(strategy, vol)
        sensitivity_results.append({
            'strategy': strategy,
            'volume': vol,
            'total_cost': costs['total_implementation_cost'],
            'cost_per_patient': costs['cost_per_patient_total']
        })

sens_df = pd.DataFrame(sensitivity_results)

# Plot sensitivity analysis
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Total cost vs volume
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[0].plot(strat_data['volume'], strat_data['total_cost'], label=strategy, linewidth=2)

ax[0].set_xlabel('Patient Volume')
ax[0].set_ylabel('Total Implementation Cost ($AUD)')
ax[0].set_title('Total Cost vs Patient Volume')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Cost per patient vs volume
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[1].plot(strat_data['volume'], strat_data['cost_per_patient'], label=strategy, linewidth=2)

ax[1].set_xlabel('Patient Volume')
ax[1].set_ylabel('Cost per Patient ($AUD)')
ax[1].set_title('Cost per Patient vs Patient Volume')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate implementation at scale
scale_scenarios = [
    {'name': 'Small Clinic (200 pts/year)', 'volume': 200},
    {'name': 'Medium Clinic (800 pts/year)', 'volume': 800},
    {'name': 'Large Center (1500 pts/year)', 'volume': 1500},
    {'name': 'Regional Hub (3000 pts/year)', 'volume': 3000}
]

scale_results = []
for scenario in scale_scenarios:
    for strategy in strategies:
        costs = calculate_implementation_costs(strategy, scenario['volume'])
        scale_results.append({
            'scenario': scenario['name'],
            'strategy': strategy,
            'volume': scenario['volume'],
            'total_cost': costs['total_implementation_cost'],
            'cost_per_patient': costs['cost_per_patient_total'],
            'providers_needed': costs['providers_needed']
        })

scale_df = pd.DataFrame(scale_results)

# Create a pivot table for analysis
cost_pivot = scale_df.pivot(index='strategy', columns='scenario', values='total_cost')

plt.figure(figsize=(14, 6))

# Heatmap of total costs by scenario and strategy
sns.heatmap(cost_pivot, annot=True, fmt='.0f', cbar_kws={'label': 'Total Cost ($AUD)'})
plt.title('Implementation Costs by Strategy and Scale')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate return on investment (ROI) timeline
def calculate_roi_timeline(strategy, patient_volume, cost_per_qaly, time_horizon=5):
    """
    Calculate the return on investment timeline for implementation.
    
    Args:
        strategy: Treatment strategy
        patient_volume: Annual patient volume
        cost_per_qaly: Cost per QALY gained from treatment
        time_horizon: Time horizon in years
        
    Returns:
        DataFrame with ROI timeline
    """
    # Calculate implementation costs
    impl_costs = calculate_implementation_costs(strategy, patient_volume * time_horizon, time_horizon)
    
    # Calculate health benefits (simplified)
    # Assume each patient gains 0.5 QALYs on average
    avg_qalys_per_patient = 0.5
    total_qalys_gained = patient_volume * time_horizon * avg_qalys_per_patient
    
    # Calculate value of health benefits
    total_health_value = total_qalys_gained * cost_per_qaly
    
    # Calculate annual benefit
    annual_patients = patient_volume
    annual_qalys = annual_patients * avg_qalys_per_patient
    annual_benefit = annual_qalys * cost_per_qaly
    
    # Calculate cumulative costs and benefits
    results = []
    cumulative_cost = impl_costs['total_startup']  # Startup costs are in year 0
    cumulative_benefit = 0
    
    # Year 0: Only startup costs
    results.append({
        'year': 0,
        'cumulative_cost': cumulative_cost,
        'cumulative_benefit': cumulative_benefit,
        'net_value': cumulative_benefit - cumulative_cost
    })
    
    # Years 1-5: Annual operations and benefits
    for year in range(1, time_horizon + 1):
        cumulative_cost += impl_costs['annual_operational_total']
        cumulative_benefit += annual_benefit
        
        results.append({
            'year': year,
            'cumulative_cost': cumulative_cost,
            'cumulative_benefit': cumulative_benefit,
            'net_value': cumulative_benefit - cumulative_cost
        })
    
    return pd.DataFrame(results)

# Calculate ROI timeline for each strategy
wtp_threshold = 50000  # Willingness-to-pay per QALY
roi_results = {}

for i, strategy in enumerate(strategies):
    roi_results[strategy] = calculate_roi_timeline(strategy, patient_volumes[i], wtp_threshold)

# Plot ROI timelines
plt.figure(figsize=(14, 8))

for strategy in strategies:
    data = roi_results[strategy]
    plt.plot(data['year'], data['cumulative_cost'], label=f'{strategy} - Costs', linestyle='--', linewidth=2)
    plt.plot(data['year'], data['cumulative_benefit'], label=f'{strategy} - Benefits', linewidth=2)
    plt.plot(data['year'], data['net_value'], label=f'{strategy} - Net Value', linestyle='-.', linewidth=2)

plt.axhline(y=0, color='black', linestyle='-', alpha=0.3)
plt.xlabel('Year')
plt.ylabel('Cumulative Value ($AUD)')
plt.title('Return on Investment Timeline by Strategy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Generate implementation recommendations
print("IMPLEMENTATION COST ANALYSIS SUMMARY")
print("="*70)

for _, row in impl_costs_df.iterrows():
    print(f"\n{row['strategy']}:\n" + "-"*30)
    print(f"  Patient Volume: {row['patient_volume']:,} patients/year")
    print(f"  Providers Needed: {int(row['providers_needed'])}")
    print(f"  Startup Costs: ${row['total_startup']:,.0f}")
    print(f"  5-Year Operational: ${row['total_5year_operational']:,.0f}")
    print(f"  Total Implementation Cost: ${row['total_implementation_cost']:,.0f}")
    print(f"  Cost per Patient: ${row['cost_per_patient_total']:,.0f}")
    
    # Calculate cost effectiveness
    avg_annual_cost = row['total_implementation_cost'] / 5  # Annualized over 5 years
    cost_per_patient_annual = avg_annual_cost / row['patient_volume']
    print(f"  Annualized Cost per Patient: ${cost_per_patient_annual:,.0f}")

print("\n\nRECOMMENDATIONS")
print("="*70)
# Find strategy with lowest cost per patient
min_cost_strategy = impl_costs_df.loc[impl_costs_df['cost_per_patient_total'].idxmin()]
print(f"1. Most Cost-Effective Implementation: {min_cost_strategy['strategy']} "
      f"(${min_cost_strategy['cost_per_patient_total']:,.0f} per patient)")

# Find strategy with lowest startup costs
min_startup_strategy = impl_costs_df.loc[impl_costs_df['total_startup'].idxmin()]
print(f"2. Lowest Capital Requirement: {min_startup_strategy['strategy']} "
      f"(${min_startup_strategy['total_startup']:,.0f} startup)")

# Calculate implementation feasibility score (0-10, higher is better)
# Lower score = more feasible
impl_costs_df['feasibility_score'] = 10 - (impl_costs_df['total_startup'] / impl_costs_df['total_startup'].max() * 5) - \
                                    (impl_costs_df['providers_needed'] / impl_costs_df['providers_needed'].max() * 5)

best_feasibility = impl_costs_df.loc[impl_costs_df['feasibility_score'].idxmax()]
print(f"3. Most Feasible Implementation: {best_feasibility['strategy']} "
      f"(Feasibility Score: {best_feasibility['feasibility_score']:.1f}/10)")

## Next Steps

1. Integrate with actual implementation data
2. Consider regional and temporal cost variations
3. Evaluate impact of implementation on treatment outcomes
4. Model scale-up and learning curve effects